# Pandas Data Wrangling

### Missing Data

Using the `GA Paid Search Traffic.csv` file from the `data` folder, let's load the "third block of data" from the file, that is, the Users' time series data, using the `pd.read_csv()` function.

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
!cat data/GA\ Paid\ Search\ Traffic.csv

# ----------------------------------------
# 1 Master View
# Paid Search Traffic
# 20190501-20190531
# ----------------------------------------

Keyword,Users,New Users,Sessions,Bounce Rate,Pages/Session,Avg. Session Duration,E-commerce Conversion Rate,Transactions,Revenue
Google Merchandise Store,"1,051",753,"1,455",27.29%,5.62,00:03:41,0.34%,5,US$256.96
(not set),546,346,759,29.78%,5.32,00:03:18,0.13%,1,US$12.99
google merch,103,82,145,28.28%,6.04,00:03:29,2.07%,3,US$86.09
google backpack,95,68,121,40.50%,4.20,00:02:27,0.00%,0,US$0.00
Google Merchandise,93,54,147,28.57%,5.37,00:03:23,0.68%,1,US$79.98
Google Apparel,92,76,127,37.80%,4.97,00:03:11,0.79%,1,US$76.37
youtuber merch,78,76,82,78.05%,1.41,00:00:06,0.00%,0,US$0.00
Google Swag,63,57,80,41.25%,5.10,00:04:14,0.00%,0,US$0.00
google merch store,54,38,69,28.99%,5.09,00:02:33,0.00%,0,US$0.00
Google Clothing,39,36,49,40.82%,3.98,00:02:43,0.00%,0,US$0.00
,"2,539","1,853","3,457",32.51%,5.21,00:03:17,0.40%,14,US$607.60

Day Index,Users

In [ ]:
ts = pd.read_csv("data/GA Paid Search Traffic.csv", skiprows=18, nrows=31)
print(ts.shape)
ts.head()

(31, 2)


,Day Index,Users
0,01/05/2019,93
1,02/05/2019,114
2,03/05/2019,121
3,04/05/2019,69
4,05/05/2019,75


There are no missing data in this time series, so let's **introduce NAs manually**. To do that, we will assign the `np.nan` object to some of the data points in the `Users` column. To make things more interesting, let's introduce a new method, `.sample()`, which **takes a sample from your DataFrame** (you can decide the size of the sample via the `n` parameter). We can use this sample to randomly select 5 rows of the DataFrame, we will then select the values in the `Users` column and replace them with the `np.nan` object.

https://stackoverflow.com/questions/28064634/random-state-pseudo-random-number-in-scikit-learn

Back before computers, they used to publish books of random numbers. random.seed() is like opening a book of random numbers to a specific page. The numbers on that page are still random, but if you remember what page you turned to you will always get the same random numbers. In python, this means your code will consistently produce the same results every time. This is useful if, for example, you a writing a tutorial for random.randint() and you want to have text that references the numbers that the function spits out (along with many types of analyses that involve randomness). If you don't care about having consistent results each time you run your code, you don't need to use it.

In [ ]:
ts.sample(n=5, random_state=42).index

Int64Index([27, 15, 23, 17, 8], dtype='int64')

In [ ]:
ts.iloc[ts.sample(n=5, random_state=42).index, 1] = np.nan

First of all, let's **look at these missing values** we just created; in order to filter a DataFrame and show just the missing values, you can use the `.isnull()` method on the DataFrame / Series itself.

In [ ]:
ts[ts['Users'].isnull()]

,Day Index,Users
8,09/05/2019,NaN
15,16/05/2019,NaN
17,18/05/2019,NaN
23,24/05/2019,NaN
27,28/05/2019,NaN


In [ ]:
ts.head(10)

,Day Index,Users
0,01/05/2019,93.0
1,02/05/2019,114.0
2,03/05/2019,121.0
3,04/05/2019,69.0
4,05/05/2019,75.0
5,06/05/2019,109.0
6,07/05/2019,140.0
7,08/05/2019,148.0
8,09/05/2019,NaN
9,10/05/2019,135.0


When you're faced with **missing values** there are several paths you can choose to follow:

- you can **drop the observations** containing missing values; this is probably the worst option, since you may be missing information, espectially if that row has values in other variables of the DataFrame
- you can **fill them with zeros**; this solution may be useful and realistic in some situations (for instance if the value was missing because there were no sales on that day) and detrimental in others (if there was a technical issue with the recording of that data point, for example).
- you can **fill them with the previous or with the next data point** (or with the mean of previous and next) if data is sequential in nature (like in our example); note that you can't apply this method if, for example, you're looking at city population and each row/observation is a different city
- you can **fill them with the average** of that column / variable
- you can do **some other kind of interpolation, prediction or filtering** via several methods such as [spline interpolation](https://en.wikipedia.org/wiki/Spline_interpolation), [moving average](https://en.wikipedia.org/wiki/Moving_average) methods and, in more complex scenarios, [linear regression](https://en.wikipedia.org/wiki/Linear_regression).

Let's see how we can fill these missing values using the `.fillna()` method:

In [ ]:
# fill with zeros
ts_fill = ts.copy() # this is a shallow copy. We will look at deep_copy later
ts_fill.fillna(0).head(10)

,Day Index,Users
0,01/05/2019,93.0
1,02/05/2019,114.0
2,03/05/2019,121.0
3,04/05/2019,69.0
4,05/05/2019,75.0
5,06/05/2019,109.0
6,07/05/2019,140.0
7,08/05/2019,148.0
8,09/05/2019,0.0
9,10/05/2019,135.0


In [ ]:
# fill with next valid observation
ts_fill.fillna(method='bfill').head(10)

,Day Index,Users
0,01/05/2019,93.0
1,02/05/2019,114.0
2,03/05/2019,121.0
3,04/05/2019,69.0
4,05/05/2019,75.0
5,06/05/2019,109.0
6,07/05/2019,140.0
7,08/05/2019,148.0
8,09/05/2019,135.0
9,10/05/2019,135.0


In [ ]:
# fill with last valid observation
ts_fill.fillna(method='ffill').head(10)

,Day Index,Users
0,01/05/2019,93.0
1,02/05/2019,114.0
2,03/05/2019,121.0
3,04/05/2019,69.0
4,05/05/2019,75.0
5,06/05/2019,109.0
6,07/05/2019,140.0
7,08/05/2019,148.0
8,09/05/2019,148.0
9,10/05/2019,135.0


In [ ]:
# fill with average of column
ts_fill.fillna(ts_fill.Users.mean()).head(10)

,Day Index,Users
0,01/05/2019,93.000000
1,02/05/2019,114.000000
2,03/05/2019,121.000000
3,04/05/2019,69.000000
4,05/05/2019,75.000000
5,06/05/2019,109.000000
6,07/05/2019,140.000000
7,08/05/2019,148.000000
8,09/05/2019,90.769231
9,10/05/2019,135.000000


Using the `.interpolate()` method, you can also fill all the missing values using a linear interpolation. Check out the [method parameter](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.interpolate.html#pandas.DataFrame.interpolate) for a list of all the options available.

In [ ]:
# fill with interpolation --> see the method parameter for more options!
ts_fill.interpolate().head(10)

,Day Index,Users
0,01/05/2019,93.0
1,02/05/2019,114.0
2,03/05/2019,121.0
3,04/05/2019,69.0
4,05/05/2019,75.0
5,06/05/2019,109.0
6,07/05/2019,140.0
7,08/05/2019,148.0
8,09/05/2019,141.5
9,10/05/2019,135.0


### Joining Data

You can connect rows in DataFrames based on one or more keys via the `pd.merge()` [function](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.merge.html). These kind of operations are analogous to **join operations** performed on a relational database using the SQL language.

We will be using the European Soccer Database, a collection of CSV files containing matches data and metadata about the teams and leagues they play in:

- match.csv
- team.csv
- leagues.csv

Let's start by loading and looking into the first two datasets:

In [ ]:
match_raw = pd.read_csv("data/European Soccer Database/match.csv")
match_raw.head()

,id,country_id,league_id,season,stage,date,match_api_id,home_team_api_id,away_team_api_id,home_team_goal,away_team_goal
0,1,1,1,2008/2009,1,2008-08-17 00:00:00,492473,9987,9993,1,1
1,2,1,1,2008/2009,1,2008-08-16 00:00:00,492474,10000,9994,0,0
2,3,1,1,2008/2009,1,2008-08-16 00:00:00,492475,9984,8635,0,3
3,4,1,1,2008/2009,1,2008-08-17 00:00:00,492476,9991,9998,5,0
4,5,1,1,2008/2009,1,2008-08-16 00:00:00,492477,7947,9985,1,3


In [ ]:
team = pd.read_csv("data/European Soccer Database/team.csv")
team.head()

,id,team_api_id,team_long_name,team_short_name
0,1,9987,KRC Genk,GEN
1,2,9993,Beerschot AC,BAC
2,3,10000,SV Zulte-Waregem,ZUL
3,4,9994,Sporting Lokeren,LOK
4,5,9984,KSV Cercle Brugge,CEB


By looking at the data, we can see that there are the following **entity relationships** that link each table to one another:

<img src="img/European Soccer DB - ERD.png" width="600">

For example, let's start by [merging](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.merge.html) the `match` with the `team` dataset via the key-pairs `home_team_api_id <-> team_api_id` and then on `away_team_api_id <-> team_api_id`:

### Merge vs Join
https://www.geeksforgeeks.org/what-is-the-difference-between-join-and-merge-in-pandas/

In [ ]:
# merge on home team; the first argument is the data being merged
match = match_raw.merge(team[['team_api_id','team_long_name']], how='left', left_on='home_team_api_id', right_on='team_api_id')
match.head()

,id,country_id,league_id,season,stage,date,match_api_id,home_team_api_id,away_team_api_id,home_team_goal,away_team_goal,team_api_id,team_long_name
0,1,1,1,2008/2009,1,2008-08-17 00:00:00,492473,9987,9993,1,1,9987,KRC Genk
1,2,1,1,2008/2009,1,2008-08-16 00:00:00,492474,10000,9994,0,0,10000,SV Zulte-Waregem
2,3,1,1,2008/2009,1,2008-08-16 00:00:00,492475,9984,8635,0,3,9984,KSV Cercle Brugge
3,4,1,1,2008/2009,1,2008-08-17 00:00:00,492476,9991,9998,5,0,9991,KAA Gent
4,5,1,1,2008/2009,1,2008-08-16 00:00:00,492477,7947,9985,1,3,7947,FCV Dender EH


In [ ]:
'''
SELECT a.*, b.*
FROM match a
LEFT JOIN team b
    on a.away_team_api_id = b.team_api_id
'''

'\nSELECT a.*, b.*\nFROM match a\nLEFT JOIN team b\n    on a.away_team_api_id = b.team_api_id\n'

In [ ]:
# merge on away team
match = match.merge(team[['team_api_id','team_long_name']], how='left', left_on='away_team_api_id', right_on='team_api_id')
match.head()

,id,country_id,league_id,season,stage,date,match_api_id,home_team_api_id,away_team_api_id,home_team_goal,away_team_goal,team_api_id_x,team_long_name_x,team_api_id_y,team_long_name_y
0,1,1,1,2008/2009,1,2008-08-17 00:00:00,492473,9987,9993,1,1,9987,KRC Genk,9993,Beerschot AC
1,2,1,1,2008/2009,1,2008-08-16 00:00:00,492474,10000,9994,0,0,10000,SV Zulte-Waregem,9994,Sporting Lokeren
2,3,1,1,2008/2009,1,2008-08-16 00:00:00,492475,9984,8635,0,3,9984,KSV Cercle Brugge,8635,RSC Anderlecht
3,4,1,1,2008/2009,1,2008-08-17 00:00:00,492476,9991,9998,5,0,9991,KAA Gent,9998,RAEC Mons
4,5,1,1,2008/2009,1,2008-08-16 00:00:00,492477,7947,9985,1,3,7947,FCV Dender EH,9985,Standard de Liège


The whole thing looks a bit messy, let's clean it up a bit by keeping just the relevant columns:

In [ ]:
match.columns

Index(['id', 'country_id', 'league_id', 'season', 'stage', 'date',
       'match_api_id', 'home_team_api_id', 'away_team_api_id',
       'home_team_goal', 'away_team_goal', 'team_api_id_x', 'team_long_name_x',
       'team_api_id_y', 'team_long_name_y'],
      dtype='object')

I **reorder some of the columns** using the double-bracket `[[ ]]` operator; note that if I omit some of the column names, they will be automatically dropped from the resulting DataFrame.

In [ ]:
# reorder columns in the DataFrame (omitting a column name will implicitly drop that column)
match = match[['match_api_id', 'league_id', 'season', 'stage', 'date',
               'home_team_api_id', 'away_team_api_id', 'team_long_name_x', 'team_long_name_y',
               'home_team_goal', 'away_team_goal']]
match.head()

,match_api_id,league_id,season,stage,date,home_team_api_id,away_team_api_id,team_long_name_x,team_long_name_y,home_team_goal,away_team_goal
0,492473,1,2008/2009,1,2008-08-17 00:00:00,9987,9993,KRC Genk,Beerschot AC,1,1
1,492474,1,2008/2009,1,2008-08-16 00:00:00,10000,9994,SV Zulte-Waregem,Sporting Lokeren,0,0
2,492475,1,2008/2009,1,2008-08-16 00:00:00,9984,8635,KSV Cercle Brugge,RSC Anderlecht,0,3
3,492476,1,2008/2009,1,2008-08-17 00:00:00,9991,9998,KAA Gent,RAEC Mons,5,0
4,492477,1,2008/2009,1,2008-08-16 00:00:00,7947,9985,FCV Dender EH,Standard de Liège,1,3


Then I **rename some of the columns** in order to make them easier to read and interpret. Check out the [documentation page](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.rename.html) of the `.rename()` method for more information.

In [ ]:
# rename columns in the DataFrame
match = match.rename(columns={"team_long_name_x": "home_team_name",
                              "team_long_name_y": "away_team_name"})
match.head()

,match_api_id,league_id,season,stage,date,home_team_api_id,away_team_api_id,home_team_name,away_team_name,home_team_goal,away_team_goal
0,492473,1,2008/2009,1,2008-08-17 00:00:00,9987,9993,KRC Genk,Beerschot AC,1,1
1,492474,1,2008/2009,1,2008-08-16 00:00:00,10000,9994,SV Zulte-Waregem,Sporting Lokeren,0,0
2,492475,1,2008/2009,1,2008-08-16 00:00:00,9984,8635,KSV Cercle Brugge,RSC Anderlecht,0,3
3,492476,1,2008/2009,1,2008-08-17 00:00:00,9991,9998,KAA Gent,RAEC Mons,5,0
4,492477,1,2008/2009,1,2008-08-16 00:00:00,7947,9985,FCV Dender EH,Standard de Liège,1,3


### Grouping data

After all this work we have a nicely shaped dataset, so it would be interesting to **investigate it further by grouping and aggregating its variables**. To do this, we can use the `.groupby()` [method](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.groupby.html) to combine all the elements of the same category and a function like `.size()`, `.sum()` or `.mean()` to aggregate the results over a specific metric.

Let's say we're interested in finding out the sum and the average home goals scored in the whole dataset; we can use the `.agg()` [method](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.agg.html):

In [ ]:
match['home_team_goal'].agg(['min', 'mean', 'sum', 'max', 'count'])

min          0.000000
mean         1.544594
sum      40127.000000
max         10.000000
count    25979.000000
Name: home_team_goal, dtype: float64

If we want to aggregate these two metrics over each `season` group, we can combine the **group by** with the aggregation method we just introduced:

In [ ]:
'''
SELECT season, min(home_team_goal), avg(home_team_goal), sum(home_team_goal), max(home_team_goal), count(home_team_goal)
FROM match
GROUP BY season
'''

'\nSELECT season, min(home_team_goal), avg(home_team_goal), sum(home_team_goal), max(home_team_goal), count(home_team_goal)\nFROM match\nGROUP BY season\n'

In [ ]:
match.groupby('season')['home_team_goal'].agg(['min', 'mean', 'sum', 'max', 'count'])

,min,mean,sum,max,count
season,,,,,
2008/2009,0,1.505412,5007,7,3326
2009/2010,0,1.541176,4978,9,3230
2010/2011,0,1.548466,5048,10,3260
2011/2012,0,1.572671,5064,8,3220
2012/2013,0,1.550000,5053,9,3260
2013/2014,0,1.578826,4787,7,3032
2014/2015,0,1.520301,5055,9,3325
2015/2016,0,1.543897,5135,10,3326


Check out [this webpage](https://pbpython.com/groupby-agg.html) for more examples on the use of groupby() and agg().

If we're interested in the number of matches played in each season (remember that in this dataset, each row represents a different match), we can use the `.size()` method, which **returns the total number of row count for each group** *(note that you don't need to specify a column, you can use it over the whole DataFrame)*:

In [ ]:
# size is the equivalent of count
match.groupby('season').size()

season
2008/2009    3326
2009/2010    3230
2010/2011    3260
2011/2012    3220
2012/2013    3260
2013/2014    3032
2014/2015    3325
2015/2016    3326
dtype: int64

If, instead, we wanted to find out the **number of matches played in each month** of every `season`, we would need to create a new `month` variable (where we [extract the month from the date](https://www.interviewqs.com/ddi-code-snippets/extract-month-year-pandas) column) and include it in the group by statement:

In [ ]:
# extract the month from the date column and save it as a new variable
match['month'] = pd.DatetimeIndex(match['date']).month
match.head()

,match_api_id,league_id,season,stage,date,home_team_api_id,away_team_api_id,home_team_name,away_team_name,home_team_goal,away_team_goal,month
0,492473,1,2008/2009,1,2008-08-17 00:00:00,9987,9993,KRC Genk,Beerschot AC,1,1,8
1,492474,1,2008/2009,1,2008-08-16 00:00:00,10000,9994,SV Zulte-Waregem,Sporting Lokeren,0,0,8
2,492475,1,2008/2009,1,2008-08-16 00:00:00,9984,8635,KSV Cercle Brugge,RSC Anderlecht,0,3,8
3,492476,1,2008/2009,1,2008-08-17 00:00:00,9991,9998,KAA Gent,RAEC Mons,5,0,8
4,492477,1,2008/2009,1,2008-08-16 00:00:00,7947,9985,FCV Dender EH,Standard de Liège,1,3,8


In [ ]:
'''
SELECT season, month, count(*)
FROM match
GROUP BY season, month
'''

'\nSELECT season, month, count(*)\nFROM match\nGROUP BY season, month\n'

In [ ]:
df1 = match.groupby(['season', 'month']).size()
df1

season     month
2008/2009  1        252
           2        343
           3        318
           4        387
           5        413
                   ... 
2015/2016  8        347
           9        322
           10       345
           11       308
           12       337
Length: 89, dtype: int64

Great, this is what we wanted! However, you may have noticed that **it returned a multi-index DataFrame**; I personally prefer working with single-index DataFrames, so if you include the `as_index=False` parameter in the `.groupby()` method, you can flatten the multi-index to a single-index DataFrame:

In [ ]:
df2 = match.groupby(['season', 'month'], as_index=False).size()
df2.head(11)

,season,month,size
0,2008/2009,1,252
1,2008/2009,2,343
2,2008/2009,3,318
3,2008/2009,4,387
4,2008/2009,5,413
5,2008/2009,7,14
6,2008/2009,8,244
7,2008/2009,9,293
8,2008/2009,10,325
9,2008/2009,11,457


Now, let's say we wanted to **find out the total number of goals** (home + away) per team in the most recent available season; we'd need to split the problem into separate subproblems:

1. first we filter the data to show just the most recent season
2. then we group all the home teams and calculate the total home goals
3. then we group all the away teams and calculate the total away goals
4. finally, we join the two DataFrames and calculate the total number of goals

In [ ]:
# find out which season is the most recent one
match.season.unique()

array(['2008/2009', '2009/2010', '2010/2011', '2011/2012', '2012/2013',
       '2013/2014', '2014/2015', '2015/2016'], dtype=object)

In [ ]:
match15_16 = match.loc[match['season']=='2015/2016'].copy()

In [ ]:
'''
SELECT home_team_name, sum(home_team_goal)
FROM match15_16
GROUP BY home_team_name
'''

'\nSELECT home_team_name, sum(home_team_goal)\nFROM match15_16\nGROUP BY home_team_name\n'

In [ ]:
# create a DataFrame that contains all the home team goals
home = match15_16.groupby(['home_team_name'], as_index=False)['home_team_goal'].sum()
home.head()

,home_team_name,home_team_goal
0,1. FC Köln,16
1,1. FSV Mainz 05,23
2,ADO Den Haag,20
3,AS Monaco,30
4,AS Saint-Étienne,25


In [ ]:
# create a DataFrame that contains all the away team goals
away = match15_16.groupby(['away_team_name'], as_index=False)['away_team_goal'].sum()
away.head()

,away_team_name,away_team_goal
0,1. FC Köln,22
1,1. FSV Mainz 05,23
2,ADO Den Haag,28
3,AS Monaco,27
4,AS Saint-Étienne,17


In [ ]:
# create a DataFrame that contains all the home + away team goals
all_goals = home.merge(away, how='inner', left_on='home_team_name', right_on='away_team_name')
all_goals.head()

,home_team_name,home_team_goal,away_team_name,away_team_goal
0,1. FC Köln,16,1. FC Köln,22
1,1. FSV Mainz 05,23,1. FSV Mainz 05,23
2,ADO Den Haag,20,ADO Den Haag,28
3,AS Monaco,30,AS Monaco,27
4,AS Saint-Étienne,25,AS Saint-Étienne,17


In [ ]:
# create a new variable containing the total number of goals (home + away)
all_goals['total_goal'] = all_goals['home_team_goal'] + all_goals['away_team_goal']
all_goals.head()

,home_team_name,home_team_goal,away_team_name,away_team_goal,total_goal
0,1. FC Köln,16,1. FC Köln,22,38
1,1. FSV Mainz 05,23,1. FSV Mainz 05,23,46
2,ADO Den Haag,20,ADO Den Haag,28,48
3,AS Monaco,30,AS Monaco,27,57
4,AS Saint-Étienne,25,AS Saint-Étienne,17,42


In [ ]:
# rename variable to team_name
all_goals = all_goals.rename(columns={"home_team_name":"team_name"})

In [ ]:
# drop away_team_name variable
all_goals.drop('away_team_name', axis=1, inplace=True)
all_goals.sort_values('total_goal', ascending=False).head()

,team_name,home_team_goal,away_team_goal,total_goal
44,FC Barcelona,67,45,112
130,Real Madrid CF,70,40,110
118,Paris Saint-Germain,59,43,102
26,Celtic,55,38,93
45,FC Basel,44,44,88


### Exercise

Perform the following tasks and /or answer to the following questions:

1. Load the `leagues.csv` dataset and call it `leagues`
2. Create a `.copy()` of the `match` dataset and call it `match_teams`
3. Join the `match_teams` and the `leagues` dataset with a left join using their common keys
4. Find out the number of matches played by each league `name` in each `season`
5. Which combination of season-league has the lowest number of matches played?

In [ ]:
# 1.

In [ ]:
leagues = pd.read_csv("data/European Soccer Database/leagues.csv")
leagues.head()

In [ ]:
# 2.

In [ ]:
match_teams = match.copy()

In [ ]:
# 3.

In [ ]:
match_teams = match_teams.merge(leagues, how='left', left_on='league_id', right_on='id')
match_teams.head()

In [ ]:
# 4.

In [ ]:
seas_leagues = match_teams.groupby(['season','name'], as_index=False).size()
seas_leagues.head()

In [ ]:
# 5.

In [ ]:
seas_leagues.sort_values('size')